# 在 Google Colab 中进行 EchoHeart 的 Qwen 微调

这个 Notebook 会自动配置环境、启动微调 (QLoRA)、运行测试、合并 LoRA 适配器并导出 GGUF 模型。

**配置**：您可以通过修改下面的第一个代码单元格中的变量来指定要使用的基础模型和数据集。

**步骤：**
1. 配置变量、克隆/更新 GitHub 仓库并定义路径。
2. 安装必要的依赖项。
3. 运行训练脚本 (QLoRA)。
4. (可选) 运行测试脚本与微调后的模型(适配器)交互。
5. 合并 LoRA 适配器到基础模型。
6. (可选) 将合并后的模型转换为 GGUF 格式。

In [ ]:
# 1. 配置变量、克隆/更新仓库并定义路径
import os

# --- 配置变量 (在此处修改！) ---
# 指定要微调的基础模型 (Hugging Face 名称或路径)
base_model_name = "Qwen/Qwen2.5-1.5B-Instruct"
# 指定数据集文件路径 (相对于仓库根目录)
dataset_file = "data/converted_dataset.json"
# QLoRA 适配器输出目录 (基于模型名称自动生成)
adapter_output_dir = f"output/{base_model_name.split('/')[-1]}-qlora-ft"
# 合并后模型的输出目录 (基于模型名称自动生成)
merged_model_output_dir = f"output/{base_model_name.split('/')[-1]}-merged-ft"
# GGUF 输出文件路径 (基于合并目录自动生成)
gguf_output_file = f"{merged_model_output_dir}/gguf-model-f16.gguf"
# ------------------------------

# -- 固定路径 --
repo_path = '/content/echoheart_demo'
# 将相对路径转换为绝对路径
adapter_path = os.path.join(repo_path, adapter_output_dir)
merged_model_path = os.path.join(repo_path, merged_model_output_dir)
dataset_abs_path = os.path.join(repo_path, dataset_file)
gguf_output_abs_path = os.path.join(repo_path, gguf_output_file) 
# --------------

print("--- 配置信息 ---")
print(f"基础模型: {base_model_name}")
print(f"数据集文件: {dataset_abs_path}")
print(f"适配器输出目录: {adapter_path}")
print(f"合并模型输出目录: {merged_model_path}")
print(f"GGUF 输出文件: {gguf_output_abs_path}")
print("------------------")

# 克隆或更新仓库
if not os.path.exists(repo_path):
  print(f"\nCloning repository into {repo_path}...")
  !git clone https://github.com/shuakami/echoheart_demo.git {repo_path}
else:
  print(f"\nRepository already exists at {repo_path}.")

%cd {repo_path}

print("Pulling latest changes...")
!git pull origin master

# 确保数据集文件存在
if not os.path.exists(dataset_abs_path):
  print(f"\n[bold red]错误：指定的数据集文件不存在: {dataset_abs_path}[/bold red]")
else:
  print(f"\n数据集文件确认存在: {dataset_abs_path}")

!pwd

In [ ]:
# 2. 安装依赖
print("Installing dependencies from requirements.txt...")
!pip install -q -r requirements.txt

print("Applying dependency fixes...")
!pip install fsspec==2024.12.0

In [ ]:
# 3. 运行训练脚本 (QLoRA)
print("Starting QLoRA training...")
!python train.py --base_model_name "{base_model_name}" --dataset_file "{dataset_file}"

## 训练完成！

微调后的模型保存在 Colab 环境文件系统的 `output/qwen-ft` 目录中。

In [ ]:
# 4. 运行测试脚本
print("Starting non-interactive testing session...")
!python test_model.py --base_model_name "{base_model_name}" --adapter_path "{adapter_path}"

## 5. (可选) 转换为 GGUF 格式

转换后的文件将尝试保存在 `{gguf_output_file}`。

In [ ]:
# 运行 LoRA 合并脚本
print("Starting LoRA merge...")
!python merge_lora.py --base_model_name "{base_model_name}" --adapter_path "{adapter_path}" --output_path "{merged_model_path}"

# 转换为 GGUF
print("Starting GGUF conversion...")
!python convert_to_gguf.py --model_dir "{merged_model_path}" --output_file "{gguf_output_abs_path}" --out_type f16